In [1]:
from bs4 import BeautifulSoup

In [2]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import json # library to handle JSON files

import requests # library to handle requests
from pandas.io.json import json_normalize

In [6]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(url).text
Canada_data = BeautifulSoup(source, 'html.parser')

In [7]:
table = Canada_data.find('table', {'class': 'wikitable sortable'})

In [10]:
column_names = ['PostalCode','Borough','Neighborhood']
canada_df = pd.read_html(str(table), skiprows=1)
canada_df = pd.DataFrame.from_dict(canada_df[0])
canada_df.columns = column_names
canada_df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [11]:
canada_df = canada_df[canada_df.Borough != 'Not assigned'].reset_index(drop=True)
canada_df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [13]:
def combine(series):
    return series.str.cat(sep=', ')

canadapcode_df = canada_df.groupby(["PostalCode", "Borough"])
canada_df = canadapcode_df.agg({'Neighborhood': combine}).reset_index()
canada_df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [14]:
def impute(row):
    if row['Neighborhood'] == 'Not assigned':
        row['Neighborhood'] = row['Borough']
    
    return row

canada_df = canada_df.apply(impute, axis=1)
canada_df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [16]:
canada_df.shape[0]

103